In [51]:
from jinja2 import Environment, PackageLoader, select_autoescape, meta
import pandas as pd
from nltk.tokenize.regexp import RegexpTokenizer

In [52]:
tknzr = RegexpTokenizer(r"\w+")

In [53]:
env = Environment(
    loader=PackageLoader('docs_module', 'templates'),
    autoescape=select_autoescape(['html', 'xml'])
)

In [55]:
mdsrc = env.loader.get_source(env, 'model.md')[0]
parsed_content = env.parse(mdsrc)
meta.find_undeclared_variables(parsed_content)

{'class_annotation_sample',
 'class_metadata_table',
 'model_author',
 'model_benchmarks',
 'model_class',
 'model_colab_url',
 'model_dataset',
 'model_date',
 'model_demo_url',
 'model_description',
 'model_language',
 'model_metadata_table',
 'model_name',
 'model_output_schema',
 'model_predicted_labels',
 'model_python_sample',
 'model_repo',
 'model_scala_sample',
 'model_tags_str',
 'model_title_seo'}

In [48]:
mdmd = env.get_template("model.md")

In [49]:
cls_meta = pd.read_csv("docs_module/metadata/class_metadata.csv")
md_meta  = pd.read_csv("docs_module/metadata/model_metadata.csv")
full_meta = pd.merge(md_meta, cls_meta, on="model_class")

In [31]:
full_meta["model_tags_str"]  = full_meta.apply(lambda x: [z.lower() for y in x for z in tknzr.tokenize(y)], axis=1)
full_meta["model_title_seo"] = full_meta.class_description +". " + full_meta.model_dataset

In [32]:
full_meta

,model_name,model_lang,upstream_deps,model_class,model_dataset,labels,reference_url,model_author,model_repo,class_description,model_tags_str,model_title_seo
0,assertion_dl,en,embeddings_clinical,AssertionDLModel,"Trained on 2010 i2b2/VA challenge on concepts,...","hypothetical, present, absent, possible, condi...",https://portal.dbmi.hms.harvard.edu/projects/n...,John Snow Labs,clinical/models,Deep Learning based Assertion of Clinical Enti...,"[assertion_dl, en, embeddings_clinical, assert...",Deep Learning based Assertion of Clinical Enti...
1,assertion_dl_large,en,embeddings_clinical,AssertionDLModel,"Trained on 2010 i2b2/VA challenge on concepts,...","hypothetical, present, absent, possible, condi...",https://portal.dbmi.hms.harvard.edu/projects/n...,John Snow Labs,clinical/models,Deep Learning based Assertion of Clinical Enti...,"[assertion_dl_large, en, embeddings_clinical, ...",Deep Learning based Assertion of Clinical Enti...


In [33]:
for i, r in full_meta.iterrows():
    with open(f"docs_module/output/{r.model_class}_{r.model_name}_{r.model_lang}","w") as f:
        f.write(mdmd.render(**r))